### Assignment : Week 3
## Efficiently finding optimal policies in MABs

In this assignment, we will work with Multi Armed Bandit environments, and try to find the best policies using different strategies to minimize the total regret.

The aim of this exercise is to code agents capable of understanding the underlying probability distributions of the environment and finding the most optimal actions as early as possible.

You can start this assignment during/after reading Grokking Ch-4.

Let's get started!

In [39]:
# importing necessary stuff
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

# if you want to use envs from Gym, import it
# import gym, gym_bandits

Let's make a simple **2-armed Bernoulli** bandit.

If you want a cleaner code, you can implement Bandits using `class` in Python.

We have included sample code for this in `bandits.py` which you can take/import.

In [40]:
# generating the underlying probability distribution

probs = np.random.random(2)

In [41]:
# our MDP is a function which takes an action and returns a reward

def mab_2_env(action):
    gen = np.random.random()

    # for bernoulli bandits, the reward is 1 if the random number is less than the probability of success, else 0
    return gen < probs[action]

Let's now make a template for testing different strategies.

Feel free to modify this code.

In [42]:
# strategy function takes in the environment function, number of actions, and a selector function
# it also takes in the number of episodes to run the strategy for (higher episodes = more accurate Q values)

def test_strategy(env, n_actions, selector, n_episodes = 1000):
    
    # initialize Q and N to 0s
    Q = np.zeros(n_actions)
    N = np.zeros(n_actions)

    # loop for n_episodes
    for e in tqdm(range(n_episodes)):
        
        # selector function takes in current Q and returns an action
        # modify the selector function according to the strategy
        action = selector(Q)

        # get the reward from the environment
        reward = env(action)

        # update N and Q
        N[action] += 1
        Q[action] += (reward - Q[action])/N[action]

    # return the best action
    return np.argmax(Q)

Implementing the simplest selector using pure-exploration strategy.

In [43]:
# selector returns a random action
random_selector = lambda Q : np.random.randint(len(Q))

Let's test this strategy.

In [44]:
test_strategy(mab_2_env, 2, random_selector)

  0%|          | 0/1000 [00:00<?, ?it/s]

0

As you can see, it returns the optimal action. Let's check if that's indeed true.

We can do that by revealing the actual `probs` distribution.

In [45]:
probs

array([0.80499935, 0.65065254])

As expected, our pure exploration strategy does indeed return the optimal action for this Bernoulli bandit. 

You can try generating new bandits with different `probs` and try out the same.

With all this in place, here's what you have to do -

Recall that, the regret $\mathcal{T}$ is given by,

$$\mathcal{T}=\sum  _{e=1} ^{E} \mathbb{E} \left[ v_* - q_* \left( A_e \right) \right]$$

We can only calculate it when we have the $v_*$ and $q_*$ functions known beforehand. Since we are making the MDPs from scratch, that's not an issue for us right now.

But remember, in real-life problems, these functions are not known. Hence we must be aware of multiple policy finding strategies and try the one which gives best results fastest.

### Todo 0

Implement the calculation of the total regret $\mathcal{T}$ for your strategy.

To do this, you will need to store the rewards obtained each episode. Modify the `strategy` function accordingly.

In [46]:
# function to calculate total regret
def regret(rewards, optimal_probs):
    T = len(rewards)
    optimal_rewards = np.max(optimal_probs)
    total_regret = optimal_rewards * T - np.sum(rewards)
    return total_regret


### Todo 1

Now, let's implement some other selection strategies and compare their regret with the simple exploration strategy.

Note that some of these strategies involve hyperparameter(s) which need to be manually adjusted. You have to play around with the values and see which one gives you best results.

This is known as "hyperparameter tuning" and is quite commonly done while working with complex models (including neural networks). Personally, you should try out some natural values (including the ones given in the book) along with some extreme values where it is easy to manually verify the correctness of your strategy.

In [47]:
# epsilon-greedy strategy
# Already implemented for you coz I am nice

def epsilon_greedy(Q, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

In [48]:
def exponentially_decaying_epsilon_greedy(Q, epsilon, episode):
    decayed_epsilon = epsilon * np.exp(-0.01 * episode)  # You can adjust the decay rate
    if np.random.random() < decayed_epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

In [49]:
def softmax_strategy(Q, temperature):
    probabilities = np.exp(Q / temperature) / np.sum(np.exp(Q / temperature))
    action = np.random.choice(len(Q), p=probabilities)
    return action

In [50]:
def ucb(Q, N, c):
    ucb_values = Q + c * np.sqrt(np.log(np.sum(N)) / (N + 1e-6))
    return np.argmax(ucb_values)

In [51]:
def thompson_sampling(Q, N, alpha, beta):
    sampled_values = np.random.beta(alpha + Q, beta + N - Q)
    return np.argmax(sampled_values)

### Todo 2

Run each strategy for 2-armed bandit environment and compare the total regrets.

You can also try plotting the regret vs episode graph and check if it matches the expected result from Grokking

In [52]:
# import plotting libraries

import matplotlib.pyplot as plt
def run_strategy(env, n_actions, selector, n_episodes=1000):
    Q = np.zeros(n_actions)
    N = np.zeros(n_actions)
    rewards = []
    optimal_rewards = []

    for _ in tqdm(range(n_episodes)):
        action = selector(Q)
        reward = env(action)
        rewards.append(reward)
        optimal_rewards.append(np.max(env(np.arange(n_actions))))

        N[action] += 1
        Q[action] += (reward - Q[action]) / N[action]

    total_regret = regret(rewards, optimal_rewards)
    return total_regret

# 2-armed Bernoulli bandit environment
np.random.seed(42)
n_actions_2_arm = 2
probs_2_arm = np.random.random(n_actions_2_arm)
env_2_arm = lambda action: np.random.random() < probs_2_arm[action]

# Run each strategy for the 2-armed Bernoulli bandit environment
n_episodes_2_arm = 1000
epsilon_2_arm = 0.1
c_ucb_2_arm = 2
alpha_thompson_2_arm = 1
beta_thompson_2_arm = 1

regrets_2_arm = {
    "Random": run_strategy(env_2_arm, n_actions_2_arm, random_selector, n_episodes_2_arm),
    "Epsilon-Greedy": run_strategy(env_2_arm, n_actions_2_arm, lambda Q: epsilon_greedy(Q, epsilon_2_arm), n_episodes_2_arm),
    "Decaying Epsilon-Greedy": run_strategy(env_2_arm, n_actions_2_arm,
                                            lambda Q: exponentially_decaying_epsilon_greedy(Q, epsilon_2_arm, _),
                                            n_episodes_2_arm),
    "Softmax": run_strategy(env_2_arm, n_actions_2_arm, lambda Q: softmax_strategy(Q, temperature=0.1), n_episodes_2_arm),
    "UCB": run_strategy(env_2_arm, n_actions_2_arm, lambda Q: ucb(Q, np.ones(n_actions_2_arm), c_ucb_2_arm),
                        n_episodes_2_arm),
    "Thompson Sampling": run_strategy(env_2_arm, n_actions_2_arm,
                                      lambda Q: thompson_sampling(Q, np.ones(n_actions_2_arm), alpha_thompson_2_arm,
                                                                  beta_thompson_2_arm),
                                      n_episodes_2_arm)
}

# Plotting
plt.figure(figsize=(10, 6))
for strategy, total_regret in regrets_2_arm.items():
    plt.plot(np.cumsum(total_regret), label=strategy)

plt.xlabel("Episodes")
plt.ylabel("Cumulative Regret")
plt.legend()
plt.title("Regret vs Episode for Different Bandit Strategies (2-armed Bernoulli bandit)")
plt.show()

### Todo 3

The 2-armed bandits might be too simple for us to actually see substantial difference in the regret of these strategies. 

Let's now create a more complicated bandit environment and replicate our results on it.

We will now implement a 10-armed Gaussian bandit. 

As required, it will have possible actions and each action will generate a reward sampled from a Gaussian distribution.

Hence, each "arm" will have a randomly generated $\mu$ and $\sigma$, and the rewards will be generated with probabilities following the $\mathcal{N}(\mu, \sigma^2)$ distribution. 

In [53]:
# 10 arm gaussian bandit

# Generate the means for each arm
means_10_arm = np.random.normal(size=n_actions_10_arm)

# Generate the variance for each arm
variances_10_arm = np.ones(n_actions_10_arm)  # You can adjust variances if needed

# Our MDP is a function which takes an action and returns a reward
def mab_10_env(action):
    # For Gaussian bandits, the reward is generated from a normal distribution
    reward = np.random.normal(loc=means_10_arm[action], scale=np.sqrt(variances_10_arm[action]))
    return reward

# Test the 10-armed Gaussian bandit environment
for _ in range(10):
    action = np.random.randint(n_actions_10_arm)
    reward = mab_10_env(action)
    print(f"Action: {action}, Reward: {reward}")


### Todo 4

Test the different strategies on the 10-armed gaussian bandit and verify your results.